In [1]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
path = '../../../../../../Desktop/Orthografietrainer2020/pickle/'

In [4]:
# import preprocessed data
infile = open('final_data_preprocessed.pkl','rb')
import_file = pickle.load(infile)
infile.close()

# balance out dataset
df1 = import_file[import_file['Erfolg'] == 0]
df2 = import_file[import_file['Erfolg'] == 1]
df1 = df1.sample(n=100000)
df2 = df2.sample(n=100000)

# append balanced dataset
import_file_small = df1.append(df2)
import_file_small.to_pickle('balanced_dataset_small_3months.pkl')

In [ ]:
smaller_features=import_file_small[['UserID','Erstloesung','Erfolg','Schwierigkeit','Wochentag','ist_Schulzeit','MehrfachFalsch','Testposition__FT', 'Testposition__nt', 'Testposition__pruefung',
       'Testposition__training', 'Testposition__version', 'Testposition__vt',
       'Testposition__zt', 'beendet', 'Fehler', 'HA__HA', 'HA__Self', 'HA__nt',
       'HA__vt', 'HA__zt', 'Klassenstufe', 'Jahredabei',
       'Sex__m', 'Sex__w']]

In [2]:
# import all sentences
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit','Art','AufgabenID']]

#Art: only GK
options = ['GK'] 
saetze = saetze[saetze['Art'].isin(options)] 

#Aufgaben ID : all except 0 (0 is kompetenztests) 
options = ['0'] 
saetze = saetze[~saetze['AufgabenID'].isin(options)] 

In [ ]:
# create empty df with null values
# rows: all userIDs from preprocessed and balanced dataset
# columns: all satzIDs

userIDs = import_file_small[['UserID']]
userIDs = userIDs.drop_duplicates()
index_col = userIDs['UserID'].tolist()
satzList = saetze["satzID"].tolist()
df = pd.DataFrame(0, index =index_col,columns =satzList)

NameError: name 'import_file_small' is not defined

In [58]:
# import xmlsaetze and xmlsaetze archiv
# important: do not only use xmlsaetze from preprocessed sentence! although we only use a balanced subset, we need all historical data

infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','SatzID','Erfolg','Datum']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)
xmlsaetze_small = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

# filter satzID: only GK and not kompetenztests
options = saetze['satzID'].tolist()
xmlsaetze_small = xmlsaetze_small[xmlsaetze_small['SatzID'].isin(options)] 

# filter unserID: only User which appear in preprocessed dataset
options = index_col
xmlsaetze_userid = xmlsaetze_small[xmlsaetze_small['UserID'].isin(options)] 


In [59]:
# add column last_login
# im live system ist das nicht notwendig, da hier der aktuelle Tag genommen wird
# beim Training der Daten muss es aber schon genommen werden, weil sonst nur Okt, Nov, Dez zum trainineren genutzt wird, obwohl User vllt im Herbst gar nicht online waren
last_login = xmlsaetze_userid[['UserID','Datum']].groupby(['UserID']).max()
last_login = pd.DataFrame(last_login.reset_index())
last_login = last_login.rename(columns={"Datum": "last_login"})

# merge last login zu datafrage
xmlsaetze_userid_last_login= xmlsaetze_userid.merge(last_login, on='UserID', how='left')

In [60]:
#iterate trough dataframe row-wise

for i in range(xmlsaetze_userid_last_login.shape[0]): #iterate over rows
    user_cell = xmlsaetze_userid_last_login.iloc[i,0]
    satz_cell = xmlsaetze_userid_last_login.iloc[i,1]
    erfolg_cell = xmlsaetze_userid_last_login.iloc[i,2]
    datum_cell = xmlsaetze_userid_last_login.iloc[i,3]
    last_login_cell = xmlsaetze_userid_last_login.iloc[i,4]

    dateOfExercise = datetime.strptime(str(datum_cell), '%Y-%m-%d %H:%M:%S').date()
    dateOfLastLogin = datetime.strptime(str(last_login_cell), '%Y-%m-%d %H:%M:%S').date()
    acceptedDate = dateOfLastLogin + pd.DateOffset(months=-3) # accepted date calculates the date of the last login minus 3 months

    #check if sentence and user is in df
    if satz_cell in df.columns:
        if user_cell in df.index:
            if(dateOfExercise > acceptedDate):
                # set new cell value
                if(erfolg_cell == 1):
                    df.loc[user_cell,satz_cell] = 1
                elif (erfolg_cell ==0):
                    df.loc[user_cell,satz_cell] = -1          

In [61]:
historical_join = df.reset_index()
historical_join = historical_join.rename(columns={"index": "UserID"})
historical_join.to_pickle('historicalJoin_3months.pkl')

In [62]:
df = pd.merge(smaller_features, historical_join, how='left', on='UserID')
df.to_pickle('smallSampleSet_3months.pkl')

In [63]:
df

,UserID,Erstloesung,Erfolg,Schwierigkeit,Wochentag,ist_Schulzeit,MehrfachFalsch,Testposition__FT,Testposition__nt,Testposition__pruefung,...,9992992,9992993,9992994,9992995,9992996,9992997,9992998,9992999,9993000,9993001
0,601013,1,0,1.672830,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,725578,1,0,1.082760,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,660886,1,0,-1.629740,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,534248,1,0,1.722470,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,691156,1,0,-0.128691,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,534342,0,1,-1.107240,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
199996,550066,1,1,-2.267480,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199997,630374,1,1,1.141720,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199998,516306,0,1,2.236460,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# infile = open('smallSampleSet_3months.pkl','rb')
# import_file = pickle.load(infile)
# infile.close()
# df = import_file

# drop duplicates
df = df.drop(columns=['UserID'])

test = df.drop_duplicates(keep='first')

test.to_pickle('FINALsmallSampleSet_3months_without_duplicates.pkl')